In [ ]:
import numpy as np
from scipy import signal
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from utils import sample_rate, sample_space as t, plot_fft_simple, plot_fft, plot_fft_and_filter_response

# Filtering the Sound

Filters help you have more control on the tone of the sound, They can attenuate or boost specific frequencies, allowing users to modify the timbre and sonic character of the synthesizers. In a lot of ways filter is one of the most identifiable parts of the sound. Depending on the filter's settings, even a single synth can change from sounding smooth and liquid to grinding and aggressive

In [ ]:
freq = 100
sine_wave = np.sin(2 * np.pi * freq * t)
square_wave = signal.square(2 * np.pi * freq * t)
sawtooth_wave = signal.sawtooth(2 * np.pi * freq * t)

### Harmonics of the waveform! 

We know how these waveforms looks like but now let us use [Fast Fourier Transform](https://en.wikipedia.org/wiki/Fast_Fourier_transform) (FFT) to see the frequency components (fundamental and harmonics) present in each wave.


In [ ]:
fig_fft, axs_fft = plt.subplots(3, 1, figsize=(10, 10), sharex=False)
fig_fft.suptitle('Spectral Content (Frequency Domain via FFT)', fontsize=16)

plot_fft(axs_fft[0], sine_wave, sample_rate, 'Sine Wave Spectrum')
plot_fft(axs_fft[1], square_wave, sample_rate, 'Square Wave Spectrum')
plot_fft(axs_fft[2], sawtooth_wave, sample_rate, 'Sawtooth Wave Spectrum')

plt.tight_layout(rect=[0, 0.03, 1, 0.96])
plt.show()

### Observations from FFT
 *   **Sine:** Shows a single peak at the fundamental frequency (freq = 100 Hz).
 *   **Square:** Shows peaks at freq (100 Hz), 3*freq (300 Hz), 5*freq (500 Hz), etc. (odd harmonics).
 *   **Sawtooth:** Shows peaks at f0 (100 Hz), 2*freq (200 Hz), 3*freq (300 Hz), etc. (all harmonics).


**Filters** help you change the balance of frequencies present in a signal. They work by **attenuating** (reducing the amplitude or "volume" of) certain frequency ranges while allowing others to **pass** through relatively unchanged.

### The Low-Pass Filter (LPF)

Removing high frequencies while allowing lower frequencies to pass

- Cutoff Frequency: The frequency at which the filter starts significantly attenuating the signal
- Filter Order: Determines how sharply the filter cuts off frequencies above the cutoff


In music this filter is typically makes the sound appear "darker," "muffled" or "smoother" because the higher, brighter harmonics are reduced or eliminated.


In [ ]:
ipd.Audio(sawtooth_wave, rate=sample_rate)

In [ ]:
cutoff_hz = 800  # Cutoff harmonics above 800 Hz.
filter_order = 101 # Higher order = sharper cutoff
normalized_cutoff = cutoff_hz / (sample_rate/2) # Normalize the cutoff frequency to Nyquist

### FIR low-pass filter (Finite Impulse Response)

In [ ]:
# Get the filter_coefficients
"""
This function computes the coefficients of a finite impulse response filter.
The filter will have linear phase
"""
filter_coefficients = signal.firwin(
    filter_order,
    normalized_cutoff,
    pass_zero='lowpass'
)
# Apply filter on filter_coefficients
"""
Zero-phase digital filtering.
It works by applying a filter to the input signal twice,
once in the forward direction and then in the reverse direction,
resulting in a filter with zero phase shift.
"""
filtered_signal = signal.filtfilt(filter_coefficients, 1.0, sawtooth_wave)

In [ ]:
ipd.Audio(filtered_signal, rate=sample_rate)

### Plots Demonstrating Filters

In [ ]:
fig_fft_comp, axs_fft_comp = plt.subplots(2, 1, figsize=(10, 8), sharex=True, sharey=True)
fig_fft_comp.suptitle(f'Spectral Comparison Before and After LPF (Cutoff={cutoff_hz}Hz)', fontsize=16)

plot_fft(axs_fft_comp[0], sawtooth_wave, freq, 'Original Spectrum', max_freq=10)
axs_fft_comp[0].axvline(cutoff_hz, color='black', linestyle='--', linewidth=1, label=f'Cutoff Freq')
axs_fft_comp[0].legend()

plot_fft(axs_fft_comp[1], filtered_signal, freq, 'Filtered Spectrum', max_freq=10)
axs_fft_comp[1].axvline(cutoff_hz, color='black', linestyle='--', linewidth=1, label=f'Cutoff Freq')
axs_fft_comp[1].legend()


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
fig_lpf, axs_lpf = plt.subplots(3, 1, figsize=(12, 10))
fig_lpf.suptitle(f'Low-Pass Filter', fontsize=20)
signal_label = 'Sawtooth Wave'
# Plot 1: Original Signal (Time Domain)
axs_lpf[0].plot(t, sawtooth_wave, label=f'Original {signal_label}', color='blue')
axs_lpf[0].set_title('Original Signal (Time Domain)')
axs_lpf[0].set_xlabel('Time [s]')
axs_lpf[0].set_ylabel('Amplitude')
axs_lpf[0].legend()
axs_lpf[0].grid(True)
num_cycles_to_show = 3
axs_lpf[0].set_xlim(0, num_cycles_to_show / freq) # Zoom time

# Plot 2: Filtered Signal (Time Domain)
axs_lpf[1].plot(t, filtered_signal, label=f'Filtered {signal_label}', color='green')
axs_lpf[1].set_title('Filtered Signal (Time Domain)')
axs_lpf[1].set_xlabel('Time [s]')
axs_lpf[1].set_ylabel('Amplitude')
axs_lpf[1].legend()
axs_lpf[1].grid(True)
axs_lpf[1].set_xlim(0, num_cycles_to_show / freq) # Zoom time

# Plot 3: Frequency Response of the Filter
w, h = signal.freqz(filter_coefficients, worN=8000) # Calculate filter frequency response
freq_hz = (w / np.pi) * (0.5 * sample_rate) # Convert frequency axis to Hz

axs_lpf[2].plot(freq_hz, 20 * np.log10(abs(h)), label='Filter Frequency Response', color='red')
axs_lpf[2].axvline(cutoff_hz, color='black', linestyle='--', linewidth=1, label=f'Cutoff Freq ({cutoff_hz} Hz)')
axs_lpf[2].axhline(-3, color='grey', linestyle=':', linewidth=1, label='-3 dB Point')
axs_lpf[2].set_title('Low-Pass Filter Frequency Response')
axs_lpf[2].set_xlabel('Frequency [Hz]')
axs_lpf[2].set_ylabel('Magnitude [dB]')
axs_lpf[2].set_ylim(-100, 5) # Adjust y-axis to see attenuation
axs_lpf[2].set_xlim(0, sample_rate/8) # Zoom frequency axis for detail near cutoff
axs_lpf[2].legend()
axs_lpf[2].grid(True)


plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## HighPass Filter

Highpass filters remove material low in the audio spectrum, while allowing higher frequencies to pass

In [ ]:
cutoff_hz_hpf = 500  # Cutoff frequency in Hz for the High-Pass Filter.
filter_order_hpf = 101 # Order of the FIR filter

normalized_cutoff_hpf = cutoff_hz_hpf / (0.5 * sample_rate)

In [ ]:
filter_coe_hpf = signal.firwin(filter_order_hpf, normalized_cutoff_hpf, pass_zero='highpass')
filtered_signal_hpf = signal.filtfilt(filter_coe_hpf, 1.0, sawtooth_wave)

In [ ]:
ipd.Audio(filtered_signal_hpf, rate=sample_rate)

In [ ]:
# --- Create Plots ---
nyquist_hpf = 0.5 * sample_rate
fig, axs = plt.subplots(2, 1, figsize=(10, 10)) # 3 rows, 1 column
fig.suptitle(f'HPF Analysis (Cutoff = {cutoff_hz_hpf} Hz)', fontsize=16, y=1.02)

# Maximum frequency to display on FFT plots (e.g., 20 times the fundamental)
max_fft_freq_display = freq * 20

# --- Plot 1: FFT of Original Signal ---
plot_fft_simple(axs[0], sawtooth_wave, sample_rate,
                f'Original sawtooth - Frequency Spectrum',
                max_fft_freq_display, line_color='dodgerblue')
axs[0].axvline(cutoff_hz_hpf, color='red', linestyle=':', linewidth=1.5, label=f'HPF Cutoff ({cutoff_hz_hpf} Hz)')
axs[0].legend(loc='upper right')

# --- Plot 2: FFT of Filtered Signal ---
plot_fft_simple(axs[1], filtered_signal_hpf, sample_rate,
                f'Filtered Sawtooth - Frequency Spectrum',
                max_fft_freq_display, line_color='green')
axs[1].axvline(cutoff_hz_hpf, color='red', linestyle=':', linewidth=1.5, label=f'HPF Cutoff ({cutoff_hz_hpf} Hz)')
axs[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

> **LPF and HPF are used quite a lot in Mixing of tracks, So a DJ might use a high-pass filter on one track to gradually reduce the bass and bring in the next track with a low-pass filter, creating a smooth transition between the two tracks**

In [ ]:
from IPython.display import IFrame
IFrame('http://www.youtube.com/embed/atVnTF4ZaTY?start=80', width=800, height=500)

### Other Filters

**BandPass:** Allow specific ranges of frequencies to pass, cutting off everything higher and lower than the specified range. For most bandpass filters, the range of frequencies allowed to pass has a fixed width. Some bandpass filters, however, provide an additional control for bandwidth—a variable control for the upper and lower cutoff boundaries

**Notch:** Allows majority of frequencies pass, except for a small portion which is cut out. Usually this type of filter is reserved for utilitarian purposes, in order to cut out specific unwanted frequencies, but it can also be used to interesting effect, especially when the cutoff frequency is varied during use.



**[E] Bandpass filter**

In [ ]:
# BandPass Filter

lowcut_hz = 100
highcut_hz = 300
filter_order = 301

**Slope**

Filters are generally designed "smooth" and musically desirable to gradually taper out harmonic material rather than imposing a strict limit on what frequency ranges are heard and not heard. The intensity with which a filter "rolls off" frequencies beyond its cutoff frequency is called slope, and is usually measured in **decibels per octave.**  The most traditional slopes are 12dB/Oct and 24dB/Oct

### Resonance

Resonance (Q) refers to a characteristic where the filter boosts or emphasizes frequencies right around its cutoff frequency.

Imagine a standard Low-Pass Filter. As you approach the cutoff frequency from below, the filter starts to attenuate higher frequencies. With resonance, instead of a smooth roll-off, there's a "bump" or a distinct peak in the filter's frequency response right at the cutoff point before the attenuation begins (or continues more steeply after the peak).


In [ ]:
cutoff_hz = 400 # Hz
fs = sample_rate
white_noise = np.random.uniform(-0.5, 0.5, int(fs * 2))
ipd.Audio(white_noise, rate=fs)

In [ ]:
fig_fft_iir, axs_fft_iir = plt.subplots(figsize=(20, 8))
N_orig = len(white_noise)
yf_orig_audio = np.abs(fft(white_noise)[0:N_orig//2]) / N_orig * 2
xf_orig_audio = fftfreq(N_orig, 1/fs)[0:N_orig//2]
axs_fft_iir.plot(xf_orig_audio, 20 * np.log10(yf_orig_audio + 1e-9), color='gray', alpha=0.7)
axs_fft_iir.set_title('Original White Noise Spectrum')

### ButterWorth Filter

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html

Unlike FIR filters Butterworth filters are **IIR (Infinite Impulse Response)**  filters

FIR filters have a finite-length impulse response, meaning their output eventually returns to zero after the input is removed. This leads to linear phase response and inherent stability. IIR filters, on the other hand, have an impulse response that theoretically extends infinitely due to feedback loops, offering advantages in computational efficiency and steeper roll-offs but potentially sacrificing linear phase and stability

In [ ]:
order_no_res = 4
# Numerator (b) and denominator (a) polynomials of the IIR filter
b_no_res, a_no_res = signal.butter(N=order_no_res, Wn=cutoff_hz, btype='low', analog=False, fs=fs)
filtered_audio_no_res = signal.lfilter(b_no_res, a_no_res, white_noise)

# Plot Spectrum
fig_fft_iir, axs_fft_iir = plt.subplots(figsize=(20, 8))
plot_fft_and_filter_response(axs_fft_iir, filtered_audio_no_res, b_no_res, a_no_res, fs,
                             f'Butterworth LPF (Order {order_no_res})', 'dodgerblue', cutoff_hz)

# Play audio
ipd.Audio(filtered_audio_no_res, rate=fs)

### Adding Low Resonance (Q=30)

The noise will still be low-passed, but you'll hear a distinct "coloration" or "emphasis" around 400 Hz.

We are using a **biquad filter** which can be used to create resonance in audio signals. The resonance is achieved by controlling the filter's coefficients, particularly the parameters related to the poles (damping factor or Q).

```
H(z) = (b0 + b1z^-1 + b2z^-2) / (a0 + a1z^-1 + a2z^-2).
```

Here the variables b0, b1, b2 (The "Feedforward" Path) and a0, a1, a2 (The "Feedback" Path) are the filter coefficients. This equation represents how the filter's own past outputs are fed back to influence the current output. This feedback is what allows for an "infinite impulse response" and enables sharp resonances and self-oscillation if the poles are close to or on the unit circle.

In [ ]:
Q_moderate = 3.0  # Quality factor for moderate resonance

# Calculate A and B for the give Q value
w0_mod = 2 * np.pi * cutoff_hz / fs
alpha_moderate = np.sin(w0_mod) / (2 * Q_moderate)

b0_mod = (1 - np.cos(w0_mod)) / 2
b1_mod = 1 - np.cos(w0_mod)
b2_mod = (1 - np.cos(w0_mod)) / 2
a0_norm_factor_mod = 1 + alpha_moderate # Denominator for normalization

b_moderate_res = np.array([b0_mod, b1_mod, b2_mod]) / a0_norm_factor_mod
a_moderate_res = np.array([a0_norm_factor_mod, -2 * np.cos(w0_mod), 1 - alpha_moderate]) / a0_norm_factor_mod

filtered_audio_moderate_res = signal.lfilter(b_moderate_res, a_moderate_res, white_noise)

# Plot Resonant LPF with the given A and B
fig_fft_iir, axs_fft_iir = plt.subplots(figsize=(20, 8))
plot_fft_and_filter_response(axs_fft_iir, filtered_audio_moderate_res, b_moderate_res, a_moderate_res, fs,
                             f'Resonant LPF (Moderate Q={Q_moderate:.1f})', 'green', cutoff_hz)

# Play audio
ipd.Audio(filtered_audio_moderate_res, rate=fs)

### Adding High Resonance (Q=10)

The LPF effect will be present, but the resonance at 400 Hz will be very prominent, possibly sounding like a "whine," "ring," or a strong "pitched tone" emerging from the noise. If Q was extremely high, you might even hear it "singing" close to self-oscillation.

In [ ]:
Q_high = 10.0 # Higher Q factor for strong resonance
w0_high = 2 * np.pi * cutoff_hz / fs
alpha_high = np.sin(w0_high) / (2 * Q_high)

b0_high = (1 - np.cos(w0_high)) / 2
b1_high = 1 - np.cos(w0_high)
b2_high = (1 - np.cos(w0_high)) / 2
a0_norm_factor_high = 1 + alpha_high

b_high_res = np.array([b0_high, b1_high, b2_high]) / a0_norm_factor_high
a_high_res = np.array([a0_norm_factor_high, -2 * np.cos(w0_high), 1 - alpha_high]) / a0_norm_factor_high

filtered_audio_high_res = signal.lfilter(b_high_res, a_high_res, white_noise)

# Plot Resonant LPF
fig_fft_iir, axs_fft_iir = plt.subplots(figsize=(20, 8))
plot_fft_and_filter_response(axs_fft_iir, filtered_audio_high_res, b_high_res, a_high_res, fs,
                             f'Resonant LPF (High Q={Q_high:.1f})', 'orangered', cutoff_hz)

# Play Audio
ipd.Audio(filtered_audio_high_res, rate=fs)

## Self Oscillation

Many filters are designed in such a way that high resonance settings result in subtle or extreme self-oscillation—that is to say, the filter itself can be made to produce a constant tone simply by virtue of its resonance being set relatively high.

when resonance is pushed to a high enough point it can effectively creating a **Sine Wave oscillator**
